In [1]:
%load_ext sql

In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import peakutils
import locate
import farwave
import time

from scipy import signal

MIC_IND_LR = {
    0: (1, 2),
    1: (2, 0),
    2: (0, 1)
}

ANGLE_OFFSET = {
    0: 0,
    1: 2.0944, # 120 deg in rad
    2: 4.18879, # 240 deg in rad
}


/home/jtmzheng/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
%matplotlib notebook

In [4]:
%sql sqlite:///schema/sunday_cardboard.db

u'Connected: None@schema/sunday_cardboard.db'

In [5]:
l = 0.3 # Length from mic to centre of array

def calc_delays(sigs, offsets):
    """ Calculate matrix of delays given N sigs
    """
    results = []
    for j in range(len(sigs)):
        for k in range(len(sigs)):
            if j != k:
                results.append((
                        (j, k), locate.xcorr_peaks(sigs[j], sigs[k], offsets[j], offsets[k], l)
                    ))
            

    # 3x3 array delays[i][j] is the delay of signal j relative to signal i
    delays = np.zeros((len(sigs), len(sigs)), dtype=np.float)
    for res in results:
        key, val = res[0], res[1]
        delays[key[0]][key[1]] = val[1] # xcorr (val, delay) tuple
        delays[key[1]][key[0]] = -delays[key[0]][key[1]]
        
    return delays

def calc_angles(sigs, delays, l):
    """ Given delays matrix, calculate angle using both farwave and solver
    """
    # Estimate "location" of sound source, create array record
    f_theta = np.deg2rad(farwave.calc_angle(delays, l))
    for j in range(len(sigs)):
        theta = 0
        if delays[j][(j+1)%3] >= 0 and delays[j][(j+2)%3] >= 0:
            lr = MIC_IND_LR[j]
            print 'Using microphone %d as closest mic - (%d left, %d right)' % (j, lr[0], lr[1])
            r, theta = locate.locate(delays[j][lr[0]], delays[j][lr[1]], l, r0=5., theta0=f_theta-ANGLE_OFFSET[j])
            theta += ANGLE_OFFSET[j]
            print 'Solver: r = %f, theta = %f' % (r, np.rad2deg(theta))
            break
    
    return np.rad2deg(theta), np.rad2deg(f_theta)


In [6]:
experiments = %sql select id, comment from experiment

Done.


In [7]:
ang_solver_raw = []
ang_solver_butter = []
ang_far_raw = []
ang_far_butter = []

for i in range(len(experiments)):
    cur_experiment_id = int(experiments[i][0])
    result = %sql select data from mic where experiment_id = :cur_experiment_id
        
    mic_0 = np.array(result[0][0])
    mic_1 = np.array(result[1][0])
    mic_2 = np.array(result[2][0])
    
    sigs = [mic_0, mic_1, mic_2]
    #try:
    try: 
        sigs_cropped, sigs_butter_cropped, offsets, _, _ = locate.crop_sigs_npeaks(sigs)
    except RuntimeError as e:
        print 'Error: %s' % str(e)
        solver_raw = solver_butter = far_raw = far_butter = 0
        ang_solver_raw.append(solver_raw)
        ang_solver_butter.append(solver_butter)
        ang_far_raw.append(far_raw)
        ang_far_butter.append(far_butter)
        continue
        
    # Calculate delays, angle using raw signal (cropped)
    delays_raw = calc_delays(sigs_cropped, offsets)
    solver_raw, far_raw = calc_angles(sigs_cropped, delays_raw, l)

    delays_butter = calc_delays(sigs_butter_cropped, offsets)
    solver_butter, far_butter = calc_angles(sigs_butter_cropped, delays_butter, l)
    #except:
    #    print 'Failure to find peaks: Setting to 0'
    #    solver_raw = far_raw = solver_butter = far_butter = 0
        
    print 'ID: %d, %f,%f,%f,%f' % (cur_experiment_id, far_raw, far_butter, solver_raw, solver_butter)
    
    ang_solver_raw.append(solver_raw)
    ang_solver_butter.append(solver_butter)
    ang_far_raw.append(far_raw)
    ang_far_butter.append(far_butter)


Done.
Angles: [4.921903055982437, 2.04854423734881, 3.4314327345753917]
Errors: [2.359307362051743, 1.8845253743438644, 1.0017960781624278]

Angle (CW): 3.317979
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 1192840.974356, theta = 419.137785


/home/jtmzheng/anaconda2/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/jtmzheng/anaconda2/lib/python2.7/site-packages/pandas/core/window.py:164: ComplexWarning: Casting complex values to real discards the imaginary part
  values = com._ensure_float64(values)


Angles: [-0.7456041413379225, 7.359936431645443, 3.7200788295409755]
Errors: [1.956077649486211, 1.6479345836283112, 1.0021115068820006]

Angle (CW): 3.784787
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 10.019682, theta = 419.999995
ID: 1, 356.682021,356.215213,419.137785,419.999995
Done.
Angles: [-19.656115406195283, 15.826988242637043, -2.830687360478364]
Errors: [1.3120385799559224, 1.3942355569254539, 1.001221660265949]

Angle (CW): -2.713323
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.390249, theta = 59.999996
Angles: [-20.872373605739075, 16.928208604482208, -2.953180362477587]
Errors: [1.2890880120049237, 1.3689280426736197, 1.0013297979954499]

Angle (CW): -2.815921
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 7.711511, theta = 65.659939
ID: 2, 2.713323,2.815921,59.999996,65.659939
Done.


/home/jtmzheng/anaconda2/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


Angles: [-19.23018976203684, 12.68822062810208, -4.701435601330439]
Errors: [1.3204122283985362, 1.4749076208826521, 1.0033760257444924]

Angle (CW): -4.560458
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 2.220533, theta = 64.166208
Angles: [-18.179664066017562, 13.00160049468454, -3.684487363428988]
Errors: [1.3418507199968275, 1.4662352644857437, 1.0020712275177526]

Angle (CW): -3.590650
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.424256, theta = 59.999996
ID: 3, 4.560458,3.590650,64.166208,59.999996
Done.
Angles: [-13.856942506075482, 37.965035479489195, 20.232518242969363]
Errors: [1.4432927216631837, 1.0788009277173092, 1.0657607444713417]

Angle (CW): 18.082354
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.560428, theta = 419.999996
Angles: [-14.89522588360019, 36.965238924494976, 18.48818703492267]
Errors: [1.4168067782035367, 1.0866404164473562, 1.054419591123336]

Angle (CW): 16.548900
Using microphone 0 as cl

In [8]:
# Assuming comment is the _actual_ angle
for i in range(len(ang_solver_raw)):
    print '%f,%f,%f,%f,%s' % (ang_far_raw[i], ang_far_butter[i], ang_solver_raw[i], ang_solver_butter[i], experiments[i][1])

356.682021,356.215213,419.137785,419.999995,0
2.713323,2.815921,59.999996,65.659939,0
4.560458,3.590650,64.166208,59.999996,0
341.917646,343.451100,419.999996,419.999996,30
341.359423,342.614617,419.999996,419.999996,30
342.679433,342.606596,422.870429,419.999996,30
293.976628,292.986597,299.999984,299.999984,60
297.745828,296.538507,299.999983,299.999983,60
296.869429,295.729009,299.999983,299.999983,60
0.000000,0.000000,0.000000,0.000000,90
258.318271,262.303953,308.140766,307.137369,90
265.685356,266.764526,299.999985,-51.536934,90
261.683272,262.812297,308.773388,306.571741,90
245.696347,245.318235,304.984795,299.999985,120
233.417018,234.981747,297.209289,298.033144,120
247.047205,248.546550,299.999984,299.999984,120
220.465965,220.801954,299.114345,299.131849,150
217.408920,215.645880,298.578267,297.743077,150
219.805948,219.689942,298.592928,298.031095,150
218.857947,219.755672,298.933712,298.955937,150
222.303833,220.682085,299.470523,298.813753,150
221.958245,221.976591,299.65